<a href="https://colab.research.google.com/gist/JRF-2018/c88a3b4ada72611a23cd6d2ede9cebd6/matrix_subs_test_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 行列変数の代入のテスト

例えばハミルトニアンから得られた等式の左辺と右辺を変形して代入に使える式を導きたい。その等式には行列が使われているとしよう。

In [1]:
from sympy import *
import traceback

In [2]:
init_printing()

In [3]:
N = Symbol("N", integer=True, positive=True)

In [4]:
M1 = MatrixSymbol("M1", N, N)

In [5]:
M2 = MatrixSymbol("M2", N, N)

In [6]:
M3 = MatrixSymbol("M3", N, N)

行列の等式が例えば次のようなものであったとする。

In [7]:
z0 = Eq((M1 ** -1) * M3, M2 ** 2)
z0

これを simplify して M3 = の形にしたい。しかし単純な simplify ではエラーが出る。

In [8]:
try: z0.simplify()
except: traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-8-10c5005b6334>", line 1, in <module>
    try: z0.simplify()
  File "/usr/local/lib/python3.7/dist-packages/sympy/core/basic.py", line 1671, in simplify
    return simplify(self, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sympy/simplify/simplify.py", line 561, in simplify
    return _eval_simplify(**kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sympy/core/relational.py", line 562, in _eval_simplify
    e = super()._eval_simplify(**kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sympy/core/relational.py", line 314, in _eval_simplify
    elif dif.equals(0):  # XXX this is expensive
  File "/usr/local/lib/python3.7/dist-packages/sympy/matrices/expressions/matexpr.py", line 386, in equals
    return self.as_explicit().equals(other)
  File "/usr/local/lib/python3.7/dist-packages/sympy/matrices/expressions/matexpr.py", line 337, in as_explicit
    'Matrix with symbolic shape '
ValueError: Matrix with symb

N を指定すると simplify できるが、望む形にはなっていない。

In [9]:
z1 = z0.subs(N, 3).simplify()
z1

両辺に M1 をかけてみる。

In [10]:
z2 = Eq(M1.subs(N, 3) * z1.rhs, M1.subs(N,3) * z1.lhs)
z2

ここで右辺を取れば M3 に代入できる形になる。

In [11]:
z2.rhs

しかし、これは rhs や lhs をやたら使う形になり美しくない。また N を具体的に指定しなければならないのも痛い。そこで Eq は最後まで取らず、z0.rhs - z0.lhs を考え、最後にそれを == 0 とすることを考える。

In [12]:
 z3 = z0.rhs - z0.lhs
 z3

In [13]:
z4 = M1 * z3
z4

これは expand する必要がある。

In [14]:
z4_ = z4.expand()
z4_

ここまでくれば M3 に代入する形は以下で出る。

In [15]:
M3_ = (z4_ + M3).doit()
M3_

問題を変えよう。次の問題を考える。z5 の等式を M3 に代入する形に変えたい。

In [16]:
z5 = Eq(M1, (M2 + M1) * M3)
z5

z3 と同じ戦略を用いる。

In [17]:
z6 = z5.rhs - z5.lhs
z6

ここに (M1 + M2) ** -1 をかけたいのであるが、expand がうまくいかない。

In [18]:
z7 = expand(((M1 + M2) ** -1) * z6)
z7

これを解くにはいったん M1 + M2 を M4 などと置く必要がある。


In [19]:
M4 = MatrixSymbol("M4", N, N)
z7_ = expand((M4 ** -1) * z6.subs(M1 + M2, M4)).subs(M4, M1 + M2)
z7_

こうなれば M3 の代入形はすぐである。

In [20]:
M3__ = (-z7_ + M3).expand().doit()
M3__